 # Dựa trên report sau về matching string:

https://gitlab.ftech.ai/computer-vision/projects/ekyc/id-card-kie/-/blob/tiendq/rlie-vocabulary/rlie_vocabulary/vocab_dev.ipynb

 # Dựa trên report sau về cách gắn tự động các instance theo một structure sẵn có sử dụng matching string:

https://gitlab.ftech.ai/computer-vision/projects/ekyc/id-card-kie/-/blob/tiendq/re_tagging_algorithm/tests/test_utils/RETAG_ALGORITHM.ipynb



In [1]:
# receive instance list from model

import pickle

with open('/home/tiendq/PycharmProjects/DeepLearningDocReconstruction/3_post-processing/example_output_for_an_image.bin', 'rb') as handle:
    boxes = pickle.load(handle)

In [36]:
import torch

id2label = {
    0: "title",
    1: "explanation",
    2: "answer",
    3: "super_title",
    4: "header",
    5: "footer",
    6: "ending",
    7: "heading",
    8: "starting"
  }

label2id= {
    "answer": 2,
    "ending": 6,
    "explanation": 1,
    "footer": 5,
    "header": 4,
    "heading": 7,
    "starting": 8,
    "super_title": 3,
    "title": 0
  }

next_of_super_title = [3, 0, 2, 1, 5, 6, 7, 8]
next_of_title = [0, 2, 1, 5, 6, 7, 8]
next_of_answer_or_explanation = [2, 1, 5, 6, 7, 8]

question_def = [0, 2, 1]
super_question = [3, question_def, 7]

merge_no_string_mark = [1, 3, 4, 5, 6, 7, 8]
merge_with_string_mark = [0, 2]

In [3]:
def prepare_boxes(data):
    list_up_left = []
    list_down_right = []
    text_list = []
    label_list = []
    for box, box_info in data.items():
        # bbox_coordinate = box
        up_left = [box[0], box[1]]
        down_right = [box[2], box[3]]
        # ocr_text = box_info
        text = box_info['original_string']
        label = box_info['box_label']

        list_up_left.append(up_left)
        list_down_right.append(down_right)
        text_list.append(text)
        label_list.append(label)

    assert len(list_up_left) == len(list_down_right)
    bb = list(zip(list_up_left, list_down_right, text_list, label_list))

    # sort bounding box vertically
    bb = sorted(bb, key=lambda x: (x[0][1]))

    return bb, list_up_left, list_down_right

In [4]:
boxes

{(183,
  118,
  377,
  140): {'token_list': [('(', 8),
   ('Đ', 8),
   ('ề', 8),
   ('thi', 8),
   ('có', 8),
   ('05', 8),
   ('trang', 8),
   (')', 8)], 'original_string': '(Đề thi có 05 trang)', 'box_label': 8},
 (159,
  344,
  659,
  365): {'token_list': [('D', 2),
   ('.', 2),
   ('Phân', 2),
   ('chia', 2),
   ('phạm', 2),
   ('vi', 2),
   ('ảnh', 2),
   ('hưởng', 2),
   ('ở', 2),
   ('châu', 2),
   ('Âu', 2),
   (',', 2),
   ('châu', 2),
   ('Á', 2),
   ('.',
    2)], 'original_string': 'D. Phân chia phạm vi ảnh hưởng ở châu Âu, châu Á.', 'box_label': 2},
 (485,
  118,
  931,
  140): {'token_list': [('Thời', 8),
   ('gian', 8),
   ('làm', 8),
   ('bài', 8),
   (':', 8),
   ('50', 8),
   ('phút', 8),
   (',', 8),
   ('không', 8),
   ('kể', 8),
   ('thời', 8),
   ('gian',
    8)], 'original_string': 'Thời gian làm bài: 50 phút, không kể thời gian', 'box_label': 8},
 (669, 138, 748, 159): {'token_list': [('phát', 8), ('để', 8)],
  'original_string': 'phát để',
  'box_label': 8},
 (

In [5]:
import numpy as np
import random

def compute_line_unit(list_up_left, list_down_right):
    """
    This function computes the line unit corresponding to two different points. The function takes two lists list_down_right and list_up_left as the input, calculates the absolute value of the difference between them (along the x-axis), takes the median of this value and returns it as a float

    :param list_down_right:
    :param list_up_left:
    :return: float
    """
    # idx = random.choice(range(len(list_down_right)))
    # sample_up_left = list_up_left[idx]
    # sample_down_right = list_down_right[idx]
    #
    # assert sample_down_right[1] - sample_up_left[1] >= 0
    # assert sample_down_right[0] - sample_up_left[0] >= 0

    line_unit = np.abs(np.array(list_down_right)- np.array(list_up_left))
    line_unit = line_unit[:,-1]
    line_unit= np.median(line_unit)
    return line_unit

In [6]:
def line_formation(bb, line_unit, overlaps_threshold=0.5):
    """
    Description
        This function implements a line formation algorithm from a given input of bounding boxes (bb). It creates a dictionary line_dict, where each value contains the line's Y-value and a list of the indexes of the elements associated with that line. The function also takes two parameters: line_unit and overlaps_threshold.

    Functionality
        The algorithm works by looping through all the elements found in the bb array. For every element, it first checks to see if line_dict is empty. If so, it adds this element as the first entry into the dictionary.

        If line_dict is not empty, the code then loops through all its entries and computes the error between the current item's y-value and those of each stored entry, taking into account the line_unit parameter. If the error is lower than the overlaps_threshold, the current item is added to the same line. Otherwise, a new line is created and populated with this item. The function then goes back to looping over all the items until it reaches the end of the array.

        At the end, the function sorts each line's items by their x-values, such that the output is sorted lexicographically.

    :param bb: list of bounding box
    :param line_unit:
    :param overlaps_threshold:
    :return:
    """
    line_dict = {}
    id_line = 0
    for idx, instance in enumerate(bb):

        if len(line_dict.items()) == 0:
            _list = [idx]
            line_dict[id_line] = (instance[0][1], _list)

        else:
            is_in_dict = False
            for key, val in line_dict.items():

                y_val, id_vals = val

                line_error = abs(instance[0][1] - y_val) / line_unit

                if line_error < overlaps_threshold:
                    id_vals.append(idx)
                    line_dict[id_line] = (min(instance[0][1],y_val), id_vals)
                    is_in_dict = True
                    break

            if is_in_dict:
                continue

            id_line+=1
            _list = [idx]
            line_dict[id_line] = (instance[0][1], _list)

    for idx, val in line_dict.items():
        list_of_content = [bb[i] for i in val[1]]
        list_of_content = sorted(list_of_content, key=lambda x: (x[0][0]))

        line_dict[idx] = val[0], list_of_content

    return line_dict

In [7]:
bb, list_up_left, list_down_right = prepare_boxes(boxes)

In [8]:
line_unit = compute_line_unit(list_up_left, list_down_right)

In [9]:
lines = line_formation(bb, line_unit)

In [10]:
lines

{0: (0,
  [([75, 0], [266, 19], 'VIET,JACK.COM', 4),
   ([342, 0], [835, 20], 'Học trực tuyến: KHOA HỌC.VIETJACK.COM', 4)]),
 1: (60,
  [([147, 60], [413, 82], 'BỘ GIÀO DỤC VÀ ĐẢO', 8),
   ([473, 60], [941, 82], 'KY THI TRUNG HỌC PHÔ THÔNG QUỐC', 8)]),
 2: (78,
  [([251, 78], [310, 98], 'TẠO', 8),
   ([625, 78], [789, 95], 'GIA NĂM 2017', 8)]),
 3: (97,
  [([160, 98], [400, 121], 'ĐỀ THI THỬ NGHIỆM', 8),
   ([508, 97], [908, 119], 'Bài thi: Khoa học xã hội;Môôn: Lịch sử', 8)]),
 4: (118,
  [([183, 118], [377, 140], '(Đề thi có 05 trang)', 8),
   ([485, 118],
    [931, 140],
    'Thời gian làm bài: 50 phút, không kể thời gian',
    8)]),
 5: (138, [([669, 138], [748, 159], 'phát để', 8)]),
 6: (180, [([630, 180], [774, 198], 'Mã đề thi (01', 8)]),
 7: (219,
  [([114, 219],
    [730, 239],
    'Họ và tên thí sinh: .......................................................................',
    8),
   ([115, 219], [305, 240], 'Họ và tên thí sinh:', 8)]),
 8: (238,
  [([114, 238],
    [707, 2

In [14]:
import pandas as pd
reading_order = []
for line_num, boxes in lines.items():
    for box in boxes[1]:
       reading_order.append(box)
line_df = pd.DataFrame.from_dict(reading_order)
# line_df = line_df.T
line_df

,0,1,2,3
0,"[75, 0]","[266, 19]","VIET,JACK.COM",4
1,"[342, 0]","[835, 20]",Học trực tuyến: KHOA HỌC.VIETJACK.COM,4
2,"[147, 60]","[413, 82]",BỘ GIÀO DỤC VÀ ĐẢO,8
3,"[473, 60]","[941, 82]",KY THI TRUNG HỌC PHÔ THÔNG QUỐC,8
4,"[251, 78]","[310, 98]",TẠO,8
5,"[625, 78]","[789, 95]",GIA NĂM 2017,8
6,"[160, 98]","[400, 121]",ĐỀ THI THỬ NGHIỆM,8
7,"[508, 97]","[908, 119]",Bài thi: Khoa học xã hội;Môôn: Lịch sử,8
8,"[183, 118]","[377, 140]",(Đề thi có 05 trang),8
9,"[485, 118]","[931, 140]","Thời gian làm bài: 50 phút, không kể thời gian",8


In [20]:
line_df['section'] = (line_df[3] != line_df[3].shift()).cumsum()
line_df

,0,1,2,3,section
0,"[75, 0]","[266, 19]","VIET,JACK.COM",4,1
1,"[342, 0]","[835, 20]",Học trực tuyến: KHOA HỌC.VIETJACK.COM,4,1
2,"[147, 60]","[413, 82]",BỘ GIÀO DỤC VÀ ĐẢO,8,2
3,"[473, 60]","[941, 82]",KY THI TRUNG HỌC PHÔ THÔNG QUỐC,8,2
4,"[251, 78]","[310, 98]",TẠO,8,2
5,"[625, 78]","[789, 95]",GIA NĂM 2017,8,2
6,"[160, 98]","[400, 121]",ĐỀ THI THỬ NGHIỆM,8,2
7,"[508, 97]","[908, 119]",Bài thi: Khoa học xã hội;Môôn: Lịch sử,8,2
8,"[183, 118]","[377, 140]",(Đề thi có 05 trang),8,2
9,"[485, 118]","[931, 140]","Thời gian làm bài: 50 phút, không kể thời gian",8,2


In [40]:
question_temp = {
    "title": [],
    "answer": [],
    "explanation": []
}

label2id = {
    "answer": 2,
    "ending": 6,
    "explanation": 1,
    "footer": 5,
    "header": 4,
    "heading": 7,
    "starting": 8,
    "super_title": 3,
    "title": 0
  }

question_list = {}
prev_id = 0
prev_section = 0
prev_type = None
current_question = question_temp.copy()
for indx, rows in line_df.iterrows():
    print(rows[3], rows['section'], rows[2])
    current_section = rows['section']
    current_type = rows[3]

    for type in question_def:
        type_name = id2label[type]
        cond_check = False
        if current_type == type:
            current_question[type_name].append(rows[2])
            cond_check = True

        if cond_check:
            break
    print(current_question)
    if current_type == question_def[-1]:
        question_list[prev_id+1]=current_question
        current_question = question_temp.copy()
        prev_id+=1



question_list



4 1 VIET,JACK.COM
{'title': [], 'answer': [], 'explanation': []}
4 1 Học trực tuyến: KHOA HỌC.VIETJACK.COM
{'title': [], 'answer': [], 'explanation': []}
8 2 BỘ GIÀO DỤC VÀ ĐẢO
{'title': [], 'answer': [], 'explanation': []}
8 2 KY THI TRUNG HỌC PHÔ THÔNG QUỐC
{'title': [], 'answer': [], 'explanation': []}
8 2 TẠO
{'title': [], 'answer': [], 'explanation': []}
8 2 GIA NĂM 2017
{'title': [], 'answer': [], 'explanation': []}
8 2 ĐỀ THI THỬ NGHIỆM
{'title': [], 'answer': [], 'explanation': []}
8 2 Bài thi: Khoa học xã hội;Môôn: Lịch sử
{'title': [], 'answer': [], 'explanation': []}
8 2 (Đề thi có 05 trang)
{'title': [], 'answer': [], 'explanation': []}
8 2 Thời gian làm bài: 50 phút, không kể thời gian
{'title': [], 'answer': [], 'explanation': []}
8 2 phát để
{'title': [], 'answer': [], 'explanation': []}
8 2 Mã đề thi (01
{'title': [], 'answer': [], 'explanation': []}
8 2 Họ và tên thí sinh: .......................................................................
{'title': [], 'answer': [

{}

In [16]:
def question_formation(lines):
    # find super question
    question_list = {}
    question_idx = 0
    for line_num in lines.keys():
        line_boxes = lines[line_num]

        for box in line_boxes[1]:
            if box[3] not in question_def:
                continue

            if box[3] == question_def[0]:
                if line_num in question_list:
                    question_list[line_num][id2label[question_def[0]]].append(box)
                else:
                    question_list[line_num]={
                        id2label[question_def[0]]: [box]
                    }




    print(question_list)


    # with questions dont belong to any superquestion

question_formation(lines)

{9: {'title': [([114, 257], [794, 278], 'Câu 1. Hội nghị Ianta (2-1945) không đưa ra quyết định nào dưới đây?', 0)]}, 14: {'title': [([113, 365], [926, 386], 'Câu 2. Ngày 24-10-1945, sau khi Quốc hội các nước thành viên phê chuẩn, bản Hiến', 0)]}, 15: {'title': [([115, 387], [373, 408], 'chương của Liên hợp quốc', 0)]}, 18: {'title': [([113, 449], [909, 471], 'Câu 3. Những năm đầu sau khi Liên Xô tan rã, Diện bang Nga thực hiện chính sách', 0)]}, 19: {'title': [([115, 471], [512, 493], 'đối ngoại ngả về phương Tây với hi vọng', 0)]}, 24: {'title': [([115, 580], [745, 601], 'Câu 4. Tại Quốc hội Mĩ (12-3-1947), Tổng thống Truman đề nghị', 0)]}, 35: {'title': [([114, 815], [926, 836], 'Câu 6. Ngoài việc giúp các nước Tây Âu phục hồi nền kinh tế, Kế hoạch Mácsan của', 0)]}, 36: {'title': [([115, 838], [598, 859], 'Mĩ (1947) còn nhằm tập hợp các nước Tây Âu vào', 0)]}}


In [ ]:
def type_of_next(current_type):
    if current_type == 3:
        return [3, 0, 2, ]

In [26]:
def merge(lines):
    reading_order = []
    for line_num, boxes in lines.items():
        for box in boxes[1]:
           reading_order.append(box)

    mrg = []
    next = []
    for box in reading_order:
        if len(next) <= 0:
            mrg.append([box])

        else:


        if box[3] == 3:
            next = next_of_super_title

        elif box[3] == 0:
            next = next_of_title

        elif box[3] in [1,2]:
            next = next_of_answer_or_explanation
    print(reading_order)

merge(lines)

[([75, 0], [266, 19], 'VIET,JACK.COM', 4), ([342, 0], [835, 20], 'Học trực tuyến: KHOA HỌC.VIETJACK.COM', 4), ([147, 60], [413, 82], 'BỘ GIÀO DỤC VÀ ĐẢO', 8), ([473, 60], [941, 82], 'KY THI TRUNG HỌC PHÔ THÔNG QUỐC', 8), ([251, 78], [310, 98], 'TẠO', 8), ([625, 78], [789, 95], 'GIA NĂM 2017', 8), ([160, 98], [400, 121], 'ĐỀ THI THỬ NGHIỆM', 8), ([508, 97], [908, 119], 'Bài thi: Khoa học xã hội;Môôn: Lịch sử', 8), ([183, 118], [377, 140], '(Đề thi có 05 trang)', 8), ([485, 118], [931, 140], 'Thời gian làm bài: 50 phút, không kể thời gian', 8), ([669, 138], [748, 159], 'phát để', 8), ([630, 180], [774, 198], 'Mã đề thi (01', 8), ([114, 219], [730, 239], 'Họ và tên thí sinh: .......................................................................', 8), ([115, 219], [305, 240], 'Họ và tên thí sinh:', 8), ([114, 238], [707, 258], 'Số báo danh:: .........................................................................', 8), ([115, 238], [251, 258], 'Số báo danh:', 8), ([114, 257], [794, 278],